In [1]:
import sys
import os
import re

parent_dir = os.path.abspath('../')  
sys.path.append(parent_dir)

from misc import *

fill_picus_results = load_data_structure("../logs/saved_ds/full_picus_results.yaml")
civer_results = load_data_structure("../logs/saved_ds/full_civer_results.yaml")
full_stats = load_data_structure("../logs/saved_ds/benchmark_stats.yaml")

new_civer_results = dict()
for k in fill_picus_results:
    new_civer_results[k] = civer_results[k]
    
civer_results = new_civer_results

In [2]:
def get_time_accuracy_results_from_logs(log_path):
    log_data = read_from_file(log_path).strip().split('\n')

    our_results = dict()

    filename = None
    result = None
    time_taken = None

    for line in log_data:
        line = line.strip()
        if line.startswith('** filename: '):
#             print((result,time_taken), filename)
            if filename is not None:

                assert filename not in our_results
                cur_stats_d = dict()
                cur_stats_d['result'] = result
                cur_stats_d['time'] = time_taken

                our_results[filename] = cur_stats_d



            filename = line.split(" ")[-1]
            result = None
            time_taken = None

        if line.startswith('** time: '):
            time_taken = float(line.split(" ")[-1])

        if line.startswith('** result: '):
            result = line.split(" ")[-1]
            if result == "CONSTRAINED!":
                result = "safe"
            elif result == "UNDER-CONSTRAINED!":
                result = "unsafe"
            else:
                assert result in ["TIMEOUT", "SURE"], result
                
                result = "unknown"


#     print((result,time_taken), filename)
    if filename is not None:

        assert filename not in our_results
        cur_stats_d = dict()
        cur_stats_d['result'] = result
        cur_stats_d['time'] = time_taken

        our_results[filename] = cur_stats_d

    return our_results



In [4]:
log_path = "../logs/ours/our_logs_111_4.log"
our_results = get_time_accuracy_results_from_logs(log_path)

all_circuit_stats = {}
for type_bm in full_stats:
    for k in full_stats[type_bm]:
        assert k not in all_circuit_stats
        all_circuit_stats[k] = full_stats[type_bm][k]
        all_circuit_stats[k]["type_bench"] = type_bm
        
for cur_d in [fill_picus_results, civer_results, our_results]:
    for k in cur_d:
        if cur_d[k]['time'] > 30:
            cur_d[k]['time'] = 30
            cur_d[k]['result'] = 'unknown'

            
overall_records_for_latex = {}
for type_metric in ['time', 'time_solved_only', 'solve_rate']:
    overall_records_for_latex[type_metric] = dict()
    for tool in ["picus", "civer", "ours"]:
        overall_records_for_latex[type_metric][tool] = dict()
        for type_bench in ['core', 'utils']:
            overall_records_for_latex[type_metric][tool][type_bench] = dict()
            for cur_category in ["small", "medium", "large", "overall"]:
                overall_records_for_latex[type_metric][tool][type_bench][cur_category] = None
            overall_records_for_latex[type_metric][tool]['both_bench'] = dict()
            
len(our_results)

176

In [5]:
print("Average time (all circuits, including TIMEOUTs):\n")

picus_total_time_type_ALL = 0
civer_total_time_type_ALL = 0
our_total_time_type_ALL = 0
num_circuits_ALL = 0
    
for type_bench in ['core', 'utils']:
    num_circuits_all = 0
    picus_total_time_type = 0
    civer_total_time_type = 0
    our_total_time_type = 0
    
    for cur_category in ["small", "medium", "large"]:
        num_circuits = 0
        picus_time = 0
        civer_time = 0
        our_time = 0
        for k in fill_picus_results:
            if all_circuit_stats[k]['category'] == cur_category and all_circuit_stats[k]['type_bench'] == type_bench:
                picus_time += fill_picus_results[k]['time'] 
                civer_time += civer_results[k]['time']
                our_time += our_results[k]['time']
                num_circuits += 1
        print(type_bench, cur_category)
        print(picus_time,civer_time,our_time)
        print(picus_time/num_circuits,civer_time/num_circuits,our_time/num_circuits)
        print(num_circuits)
        picus_total_time_type += picus_time
        civer_total_time_type += civer_time
        our_total_time_type += our_time
        num_circuits_all += num_circuits
        overall_records_for_latex["time"]["picus"][type_bench][cur_category] = picus_time/num_circuits
        overall_records_for_latex["time"]["civer"][type_bench][cur_category] = civer_time/num_circuits
        overall_records_for_latex["time"]["ours"][type_bench][cur_category] = our_time/num_circuits
    
    print(type_bench, "all") 
    print(picus_total_time_type,civer_total_time_type,our_total_time_type)
    print(picus_total_time_type/num_circuits_all,civer_total_time_type/num_circuits_all,our_total_time_type/num_circuits_all)
    print(num_circuits_all)
    print()
    
    picus_total_time_type_ALL += picus_total_time_type
    civer_total_time_type_ALL += civer_total_time_type
    our_total_time_type_ALL += our_total_time_type
    num_circuits_ALL += num_circuits_all
    
    cur_category = "overall"
    overall_records_for_latex["time"]["picus"][type_bench][cur_category] = picus_total_time_type/num_circuits_all
    overall_records_for_latex["time"]["civer"][type_bench][cur_category] = civer_total_time_type/num_circuits_all
    overall_records_for_latex["time"]["ours"][type_bench][cur_category] = our_total_time_type/num_circuits_all
     
        
print("all bench", "all") 
print(picus_total_time_type_ALL,civer_total_time_type_ALL,our_total_time_type_ALL)
print(picus_total_time_type_ALL/num_circuits_ALL,civer_total_time_type_ALL/num_circuits_ALL,our_total_time_type_ALL/num_circuits_ALL)
print(num_circuits_ALL)

type_bench = "both_bench"
overall_records_for_latex["time"]["picus"][type_bench] = picus_total_time_type_ALL/num_circuits_ALL
overall_records_for_latex["time"]["civer"][type_bench] = civer_total_time_type_ALL/num_circuits_ALL
overall_records_for_latex["time"]["ours"][type_bench] = our_total_time_type_ALL/num_circuits_ALL


Average time (all circuits, including TIMEOUTs):

core small
578.5674459999999 78.46701502799988 119.64373099999997
9.183610253968252 1.2455081750476171 1.8991068412698409
63
core medium
492.07106799999997 132.22589802742004 165.50604699999997
20.502961166666665 5.509412417809169 6.896085291666665
24
core large
621.7893690000001 352.2406280040741 487.11706899999996
27.03432039130435 15.314809913220612 21.179002999999998
23
core all
1692.4278829999998 562.933541059494 772.2668469999999
15.385708027272726 5.1175776459954 7.0206076999999985
110

utils small
395.12582699999996 65.56728911399841 172.60802800000005
8.0637923877551 1.3381079411020085 3.522612816326532
49
utils medium
125.40851800000002 48.949421644210815 60.175084999999996
15.676064750000002 6.118677705526352 7.5218856249999995
8
utils large
202.951681 193.82696175575256 106.92262099999999
22.55018677777778 21.536329083972507 11.88029122222222
9
utils all
723.486026 308.3436725139618 339.705734
10.961909484848485 4.6718738259

In [6]:
print("Average time (solved circuits):\n")

picus_total_time_type_ALL = 0
civer_total_time_type_ALL = 0
our_total_time_type_ALL = 0
num_circuits_ALL = 0

for type_bench in ['core', 'utils']:
    num_circuits_all = 0
    picus_total_time_type = 0
    civer_total_time_type = 0
    our_total_time_type = 0
    
    for cur_category in ["small", "medium", "large"]:
        num_circuits = 0
        picus_time = 0
        civer_time = 0
        our_time = 0
        for k in fill_picus_results:
            if all_circuit_stats[k]['category'] == cur_category and all_circuit_stats[k]['type_bench'] == type_bench:
                if fill_picus_results[k]['result'] != 'unknown':
                    picus_time += fill_picus_results[k]['time'] 
                if civer_results[k]['result'] != 'unknown':
                    civer_time += civer_results[k]['time']
                if our_results[k]['result'] != 'unknown':
                    our_time += our_results[k]['time']
                num_circuits += 1
        print(type_bench, cur_category)
        print(picus_time,civer_time,our_time)
        print(picus_time/num_circuits,civer_time/num_circuits,our_time/num_circuits)
        picus_total_time_type += picus_time
        civer_total_time_type += civer_time
        our_total_time_type += our_time
        num_circuits_all += num_circuits
        overall_records_for_latex["time_solved_only"]["picus"][type_bench][cur_category] = picus_time/num_circuits
        overall_records_for_latex["time_solved_only"]["civer"][type_bench][cur_category] = civer_time/num_circuits
        overall_records_for_latex["time_solved_only"]["ours"][type_bench][cur_category] = our_time/num_circuits
    
    print(type_bench, "all") 
    print(picus_total_time_type,civer_total_time_type,our_total_time_type)
    print(picus_total_time_type/num_circuits_all,civer_total_time_type/num_circuits_all,our_total_time_type/num_circuits_all)
    print()
    
    picus_total_time_type_ALL += picus_total_time_type
    civer_total_time_type_ALL += civer_total_time_type
    our_total_time_type_ALL += our_total_time_type
    num_circuits_ALL += num_circuits_all
    
    cur_category = "overall"
    overall_records_for_latex["time_solved_only"]["picus"][type_bench][cur_category] = picus_total_time_type/num_circuits_all
    overall_records_for_latex["time_solved_only"]["civer"][type_bench][cur_category] = civer_total_time_type/num_circuits_all
    overall_records_for_latex["time_solved_only"]["ours"][type_bench][cur_category] = our_total_time_type/num_circuits_all
    
     
        
print("all bench", "all") 
print(picus_total_time_type_ALL,civer_total_time_type_ALL,our_total_time_type_ALL)
print(picus_total_time_type_ALL/num_circuits_ALL,civer_total_time_type_ALL/num_circuits_ALL,our_total_time_type_ALL/num_circuits_ALL)
type_bench = "both_bench"
overall_records_for_latex["time_solved_only"]["picus"][type_bench] = picus_total_time_type_ALL/num_circuits_ALL
overall_records_for_latex["time_solved_only"]["civer"][type_bench] = civer_total_time_type_ALL/num_circuits_ALL
overall_records_for_latex["time_solved_only"]["ours"][type_bench] = our_total_time_type_ALL/num_circuits_ALL


Average time (solved circuits):

core small
263.61092599999984 11.457225799560547 47.07924899999999
4.18430041269841 0.18186072697715153 0.7472896666666665
core medium
102.071068 4.4385902881622314 17.961892000000002
4.252961166666666 0.18494126200675964 0.7484121666666668
core large
21.789369 0.41120076179504395 40.96296
0.9473638695652175 0.017878293991088867 1.7809982608695654
core all
387.4713629999998 16.307016849517822 106.00410099999999
3.5224669363636347 0.1482456077228893 0.9636736454545454

utils small
229.41851700000004 4.454735994338989 47.150791
4.682010551020409 0.0909129794763059 0.9622610408163265
utils medium
35.408518 1.2390005588531494 8.404962999999999
4.42606475 0.15487506985664368 1.0506203749999998
utils large
22.951681 0 46.922621
2.550186777777778 0.0 5.213624555555556
utils all
287.77871600000003 5.693736553192139 102.478375
4.360283575757577 0.08626873565442635 1.5527026515151514

all bench all
675.2500789999999 22.00075340270996 208.482476
3.836648176136363 

In [7]:
print("Solved rate:\n")

num_circuits_ALL = 0
picus_safe_ALL = 0
civer_safe_ALL = 0
our_safe_ALL = 0

picus_unsafe_ALL = 0
civer_unsafe_ALL = 0
our_unsafe_ALL = 0

picus_unknown_ALL = 0
civer_unknown_ALL = 0
our_unknown_ALL = 0



for type_bench in ['core', 'utils']:
    num_circuits_all = 0
    picus_safe_all = 0
    civer_safe_all = 0
    our_safe_all = 0

    picus_unsafe_all = 0
    civer_unsafe_all = 0
    our_unsafe_all = 0

    picus_unknown_all = 0
    civer_unknown_all = 0
    our_unknown_all = 0
    
    for cur_category in ["small", "medium", "large"]:
        num_circuits = 0
        picus_safe = 0
        civer_safe = 0
        our_safe = 0
        
        picus_unsafe = 0
        civer_unsafe = 0
        our_unsafe = 0
        
        picus_unknown = 0
        civer_unknown = 0
        our_unknown = 0
        for k in fill_picus_results:
            if all_circuit_stats[k]['category'] == cur_category and all_circuit_stats[k]['type_bench'] == type_bench:
                if fill_picus_results[k]['result'] == 'safe':
                    picus_safe += 1
                elif fill_picus_results[k]['result'] == 'unsafe':
                    picus_unsafe += 1
                else:
                    assert fill_picus_results[k]['result'] == 'unknown'
                    picus_unknown += 1
                    
                if civer_results[k]['result'] == 'safe':
                    civer_safe += 1
                elif civer_results[k]['result'] == 'unsafe':
                    civer_unsafe += 1
                else:
                    assert civer_results[k]['result'] == 'unknown'
                    civer_unknown += 1
                    
                if our_results[k]['result'] == 'safe':
                    our_safe += 1
                elif our_results[k]['result'] == 'unsafe':
                    our_unsafe += 1
                else:
                    assert our_results[k]['result'] == 'unknown'
                    our_unknown += 1
                    
                num_circuits += 1
        print(type_bench, cur_category)
        print(picus_safe,civer_safe,our_safe)
        print(picus_unsafe,civer_unsafe,our_unsafe)
        print(picus_unknown,civer_unknown,our_unknown)
        print(1-picus_unknown/num_circuits,1-civer_unknown/num_circuits,1-our_unknown/num_circuits)
        print()
        num_circuits_all += num_circuits
        picus_safe_all +=picus_safe
        civer_safe_all +=civer_safe
        our_safe_all +=our_safe

        picus_unsafe_all +=picus_unsafe
        civer_unsafe_all +=civer_unsafe
        our_unsafe_all +=our_unsafe

        picus_unknown_all +=picus_unknown
        civer_unknown_all +=civer_unknown
        our_unknown_all +=our_unknown
        
        picus_rate = (1 - picus_unknown / num_circuits) * 100
        civer_rate = (1 - civer_unknown / num_circuits) * 100
        our_rate = (1 - our_unknown / num_circuits) * 100

        
        overall_records_for_latex["solve_rate"]["picus"][type_bench][cur_category] = picus_rate
        overall_records_for_latex["solve_rate"]["civer"][type_bench][cur_category] = civer_rate
        overall_records_for_latex["solve_rate"]["ours"][type_bench][cur_category] = our_rate
    
        
        
        
    print(type_bench, "all") 
    print(picus_safe_all,civer_safe_all,our_safe_all)
    print(picus_unsafe_all,civer_unsafe_all,our_unsafe_all)
    print(picus_unknown_all,civer_unknown_all,our_unknown_all)
    print(1-picus_unknown_all/num_circuits_all,1-civer_unknown_all/num_circuits_all,1-our_unknown_all/num_circuits_all)
    print()
        
    num_circuits_ALL += num_circuits_all
    picus_safe_ALL += picus_safe_all
    civer_safe_ALL += civer_safe_all
    our_safe_ALL += our_safe_all

    picus_unsafe_ALL += picus_unsafe_all
    civer_unsafe_ALL += civer_unsafe_all
    our_unsafe_ALL += our_unsafe_all

    picus_unknown_ALL += picus_unknown_all
    civer_unknown_ALL += civer_unknown_all
    our_unknown_ALL += our_unknown_all

    picus_rate_all = (1 - picus_unknown_all / num_circuits_all) * 100
    civer_rate_all = (1 - civer_unknown_all / num_circuits_all) * 100
    our_rate_all = (1 - our_unknown_all / num_circuits_all) * 100
    
    cur_category = "overall"
    overall_records_for_latex["solve_rate"]["picus"][type_bench][cur_category] = picus_rate_all
    overall_records_for_latex["solve_rate"]["civer"][type_bench][cur_category] = civer_rate_all
    overall_records_for_latex["solve_rate"]["ours"][type_bench][cur_category] = our_rate_all
     
    
    
    
print("all bench", "all") 
print(picus_safe_ALL,civer_safe_ALL,our_safe_ALL)
print(picus_unsafe_ALL,civer_unsafe_ALL,our_unsafe_ALL)
print(picus_unknown_ALL,civer_unknown_ALL,our_unknown_ALL)
print(1-picus_unknown_ALL/num_circuits_ALL,1-civer_unknown_ALL/num_circuits_ALL,1-our_unknown_ALL/num_circuits_ALL)


picus_rate_ALL = (1 - picus_unknown_ALL / num_circuits_ALL) * 100
civer_rate_ALL = (1 - civer_unknown_ALL / num_circuits_ALL) * 100
our_rate_ALL = (1 - our_unknown_ALL / num_circuits_ALL) * 100


type_bench = "both_bench"
overall_records_for_latex["solve_rate"]["picus"][type_bench] = picus_rate_ALL
overall_records_for_latex["solve_rate"]["civer"][type_bench] = civer_rate_ALL
overall_records_for_latex["solve_rate"]["ours"][type_bench] = our_rate_ALL


Solved rate:

core small
46 48 54
6 5 6
11 10 3
0.8253968253968254 0.8412698412698413 0.9523809523809523

core medium
11 5 17
0 0 0
13 19 7
0.45833333333333337 0.20833333333333337 0.7083333333333333

core large
3 1 8
0 0 0
20 22 15
0.13043478260869568 0.04347826086956519 0.34782608695652173

core all
60 54 79
6 5 6
44 51 25
0.6 0.5363636363636364 0.7727272727272727

utils small
38 36 39
5 5 5
6 8 5
0.8775510204081632 0.8367346938775511 0.8979591836734694

utils medium
5 1 5
0 0 1
3 7 2
0.625 0.125 0.75

utils large
3 0 7
0 0 0
6 9 2
0.33333333333333337 0.0 0.7777777777777778

utils all
46 37 51
5 5 6
15 24 9
0.7727272727272727 0.6363636363636364 0.8636363636363636

all bench all
106 91 130
11 10 12
59 75 34
0.6647727272727273 0.5738636363636364 0.8068181818181819


In [8]:
overall_records_for_latex

{'time': {'picus': {'core': {'small': 9.183610253968252,
    'medium': 20.502961166666665,
    'large': 27.03432039130435,
    'overall': 15.385708027272726},
   'both_bench': 13.726783573863635,
   'utils': {'small': 8.0637923877551,
    'medium': 15.676064750000002,
    'large': 22.55018677777778,
    'overall': 10.961909484848485}},
  'civer': {'core': {'small': 1.2455081750476171,
    'medium': 5.509412417809169,
    'large': 15.314809913220612,
    'overall': 5.1175776459954},
   'both_bench': 4.9504387134855445,
   'utils': {'small': 1.3381079411020085,
    'medium': 6.118677705526352,
    'large': 21.536329083972507,
    'overall': 4.671873825969118}},
  'ours': {'core': {'small': 1.8991068412698409,
    'medium': 6.896085291666665,
    'large': 21.179002999999998,
    'overall': 7.0206076999999985},
   'both_bench': 6.318026028409091,
   'utils': {'small': 3.522612816326532,
    'medium': 7.5218856249999995,
    'large': 11.88029122222222,
    'overall': 5.147056575757576}}},
 

In [9]:
import numpy as np

def create_latex_table(data):
    sizes = ['small', 'medium', 'large', 'overall']
    benchmarks = ['utils', 'core']

    latex_table = [
        "\\begin{table*}[htbp]",
        "\\centering",
        "\\begin{tabular}{|l|l||cccc||cccc||c|}",
        "\\hline",
        "\\textbf{Metric} & \\textbf{Category} & $\\mathcal{U}_{small}$ & $\\mathcal{U}_{medium}$ & $\\mathcal{U}_{large}$ & $\\mathcal{U}_{overall}$ & $\\mathcal{C}_{small}$ & $\\mathcal{C}_{medium}$ & $\\mathcal{C}_{large}$ & $\\mathcal{C}_{overall}$ & \\textbf{Overall} \\\\",
        "\\hline \\hline",
        "\\multirow{4}{*}{\\textbf{Variables (\\#)}} & In & 5 & 98 & 145 & 35 & 27 & 39 & 146 & 54 & 47 \\\\",
        " \\cline{2-11}",
        " & Out & 2 & 2 & 57 & 9 & 10 & 78 & 36 & 30 & 22 \\\\",
        " \\cline{2-11}",
        "  & Witness & 13 & 343 & 14,165 & 1,983 & 11 & 397 & 32,579 & 6,905 & 5,059 \\\\",
        " \\cline{2-11}",
        "  & Overall  & 19 & 443 & 14,367 & 2,027 & 48 & 513 & 32,761 & 6,989 & 5,129 \\\\"
        "\\hline \\hline",
        "\\multirow{3}{*}{\\textbf{Constraints (\\#)}} & Linear  & 7 & 181 & 8,701 & 1,214 & 5 & 200 &  26,116 &  5,507 &  3,897\\\\"
        " \\cline{2-11}",
        " & Non-linear  & 7 & 163 & 5,529 & 779 & 12 & 274 & 6,540 & 1,434 & 1,188 \\\\",
        " \\cline{2-11}",
        "  & Overall & 14 & 344 & 14,230 & 1,993 & 17 & 474 & 32,657 & 6,941 & 5,086  \\\\",
        "\\hline \\hline"
        
    ]


    metrics = ['time', 'solve_rate']


    for metric in metrics:
        first_row_of_metric = True
        method_count = len(data[metric])
        for method in data[metric].keys():
            row = []
            if first_row_of_metric:
                cur_metric = metric.replace('_', '\_')
                if metric == 'time':
                    cur_metric = "\\textbf{Time (s)}"
                else:
                    cur_metric = "\\textbf{Solved (\\%)}"
                row.append(f"\\multirow{{{method_count}}}{{*}}{{{cur_metric}}}")
                first_row_of_metric = False
            else:
                row.append("")
            
            if method == 'ours':
                row.append("ConsCS")
            elif method == 'picus':
                row.append('Picus~\\cite{yanju2023automated}')
            else:
                assert method == 'civer'
                row.append('Civer~\\cite{rubio2024scalable}')

            for benchmark in benchmarks:
                for size in sizes:
                    value = data[metric][method][benchmark][size]
                    formatted_value = f"{value:.2f} s" if metric != 'solve_rate' else f"{value:.2f} \\%"
                    if method == 'ours' and metric == 'solve_rate':
                        formatted_value = f"\\textbf{{{formatted_value}}}"
                    row.append(formatted_value)
                    


            value = data[metric][method]['both_bench']
            formatted_value = f"{value:.2f} s" if metric != 'solve_rate' else f"{value:.2f} \\%"
            if method == 'ours' and metric == 'solve_rate':
                formatted_value = f"\\textbf{{{formatted_value}}}"
            row.append(formatted_value)
                    
                    
                    
            latex_table.append(" & ".join(row) + " \\\\" + "\n \\cline{2-11}")
            if method == list(data[metric].keys())[-1]:
                latex_table.append("\\hline \\hline")

    latex_table.extend([
        "\\end{tabular}",
        "\\caption{The performance metrics across different benchmarks and sizes. \\(\\mathcal{U}\\) and \\(\\mathcal{C}\\) represent the \\texttt{utils} and  the \\texttt{core} benchmark, respectively. Subscripts represent sizes of the dataset. The rightmost column '\\textbf{Overall}' shows the statistics over all circuits.}",
        "\\label{tab:performance_metrics}",
        "\\end{table*}"
    ])

    return "\n".join(latex_table)

latex_code = create_latex_table(overall_records_for_latex)
print(latex_code)

\begin{table*}[htbp]
\centering
\begin{tabular}{|l|l||cccc||cccc||c|}
\hline
\textbf{Metric} & \textbf{Category} & $\mathcal{U}_{small}$ & $\mathcal{U}_{medium}$ & $\mathcal{U}_{large}$ & $\mathcal{U}_{overall}$ & $\mathcal{C}_{small}$ & $\mathcal{C}_{medium}$ & $\mathcal{C}_{large}$ & $\mathcal{C}_{overall}$ & \textbf{Overall} \\
\hline \hline
\multirow{4}{*}{\textbf{Variables (\#)}} & In & 5 & 98 & 145 & 35 & 27 & 39 & 146 & 54 & 47 \\
 \cline{2-11}
 & Out & 2 & 2 & 57 & 9 & 10 & 78 & 36 & 30 & 22 \\
 \cline{2-11}
  & Witness & 13 & 343 & 14,165 & 1,983 & 11 & 397 & 32,579 & 6,905 & 5,059 \\
 \cline{2-11}
  & Overall  & 19 & 443 & 14,367 & 2,027 & 48 & 513 & 32,761 & 6,989 & 5,129 \\\hline \hline
\multirow{3}{*}{\textbf{Constraints (\#)}} & Linear  & 7 & 181 & 8,701 & 1,214 & 5 & 200 &  26,116 &  5,507 &  3,897\\ \cline{2-11}
 & Non-linear  & 7 & 163 & 5,529 & 779 & 12 & 274 & 6,540 & 1,434 & 1,188 \\
 \cline{2-11}
  & Overall & 14 & 344 & 14,230 & 1,993 & 17 & 474 & 32,657 & 6,941 &

In [10]:
for category in [ 'utils', 'core']:
    for size in ['small', 'medium', 'large']:
        in_sum = 0
        out_sum = 0
        witness_sum = 0
        
        linear_sum = 0
        nonlinear_sum = 0
        
        count = 0
        for k in all_circuit_stats:
            if all_circuit_stats[k]["type_bench"] == category and all_circuit_stats[k]["category"] == size:
                in_sum += all_circuit_stats[k]["total inputs"]
                out_sum += all_circuit_stats[k]["total outputs"]
                witness_sum += all_circuit_stats[k]["witness"]

                linear_sum += all_circuit_stats[k]["linear constraints"]
                nonlinear_sum += all_circuit_stats[k]["non-linear constraints"]

                count += 1
        print(category, size, round(in_sum/count), round(out_sum/count), round(witness_sum/count), round((in_sum + out_sum + witness_sum)/count), "||||" ,round(linear_sum/count), round(nonlinear_sum/count), round((linear_sum + nonlinear_sum)/count))
                                                                                              
                                                                                              
                                                                                              

utils small 5 2 13 19 |||| 7 7 14
utils medium 98 2 343 443 |||| 181 163 344
utils large 145 57 14165 14367 |||| 8701 5529 14230
core small 27 10 11 48 |||| 5 12 17
core medium 39 78 397 513 |||| 200 274 474
core large 146 36 32579 32761 |||| 26116 6540 32657


In [11]:
for category in [ 'utils', 'core']:
    in_sum = 0
    out_sum = 0
    witness_sum = 0

    linear_sum = 0
    nonlinear_sum = 0
    count = 0

    for size in ['small', 'medium', 'large']:
        for k in all_circuit_stats:
            if all_circuit_stats[k]["type_bench"] == category and all_circuit_stats[k]["category"] == size:
                in_sum += all_circuit_stats[k]["total inputs"]
                out_sum += all_circuit_stats[k]["total outputs"]
                witness_sum += all_circuit_stats[k]["witness"]

                linear_sum += all_circuit_stats[k]["linear constraints"]
                nonlinear_sum += all_circuit_stats[k]["non-linear constraints"]

                count += 1
    print(category, size, round(in_sum/count), round(out_sum/count), round(witness_sum/count), round((in_sum + out_sum + witness_sum)/count), "||||" ,round(linear_sum/count), round(nonlinear_sum/count), round((linear_sum + nonlinear_sum)/count))

                                                                                              
                                                                                              

utils large 35 9 1983 2027 |||| 1214 779 1993
core large 54 30 6905 6989 |||| 5507 1434 6941


In [12]:
in_sum = 0
out_sum = 0
witness_sum = 0

linear_sum = 0
nonlinear_sum = 0
count = 0
for category in [ 'utils', 'core']:


    for size in ['small', 'medium', 'large']:
        for k in all_circuit_stats:
            if all_circuit_stats[k]["type_bench"] == category and all_circuit_stats[k]["category"] == size:
                in_sum += all_circuit_stats[k]["total inputs"]
                out_sum += all_circuit_stats[k]["total outputs"]
                witness_sum += all_circuit_stats[k]["witness"]

                linear_sum += all_circuit_stats[k]["linear constraints"]
                nonlinear_sum += all_circuit_stats[k]["non-linear constraints"]

                count += 1
print(category, size, round(in_sum/count), round(out_sum/count), round(witness_sum/count), round((in_sum + out_sum + witness_sum)/count), "||||" ,round(linear_sum/count), round(nonlinear_sum/count), round((linear_sum + nonlinear_sum)/count))
       

core large 47 22 5059 5129 |||| 3897 1188 5086
